***This notebook contains a script that sends Student Contracts to Students for signing***

**Add local library to path**

In [150]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path + '/local_library')

***Import Libraries***

In [151]:
import pandas as pd
import requests
import json
import re
from datetime import date
from local_library import import_worksheet
from local_library import export_worksheet
from local_library import send_bulk_signrequest

**Import Data**

In [152]:
#import 2020 data worksheet
cohort_df = import_worksheet("Cohort 2021 Data", "Cohort 2021 Data")
#cohort_df = cohort_df[cohort_df['Active / Not active/Early absorption'] == 'Active']


**Create Learner FullName**

In [153]:
cohort_df['middle_name'] = cohort_df['Middle Name(s)'].fillna("")
cohort_df['learner_name'] = cohort_df['First Name'].str.strip().str.lower() + " " + cohort_df['middle_name'].str.strip().str.lower()
cohort_df['learner_name'] = cohort_df['learner_name'].str.strip().str.title()
cohort_df['learner_fullname'] = cohort_df['learner_name'].str.strip().str.lower() + " " + cohort_df['Last Name'].str.strip().str.lower()
cohort_df['learner_fullname'] = [re.sub(' +', ' ', fullname) for fullname in cohort_df['learner_fullname']]
cohort_df['learner_fullname'] = cohort_df['learner_fullname'].str.title()
cohort_df['learner_initials'] = [''.join([x[0].upper() for x in fullname.split(' ')]) for fullname in cohort_df['learner_fullname']]


**Create Learner Passport**

In [154]:
cohort_df["learner_passport"] = [passport if len(str(passport)) != 13 else " " for passport in cohort_df["ID / Passport Number"]]


**Create Learner Equity Code**

In [155]:
cohort_df['learner_equity'] = cohort_df['Ethnicity'].map({'White':'WH', 'white':'WH', 'Coloured':'BC', 'coloured':'BC', 
                                                     'black':'BA', 'Black':'BA', 'Asian':'BI', 'indian':'BI','Indian':'BI'})

**Create Learner Learnership**

In [156]:
cohort_df['learner_nationality'] = cohort_df['Nationality'].map({'South Africa':'SA', 'Zimbabwe':'ZIM', 'Congolese':'ROA',
                                                                'Dutch':'EUR', 'Nigerian':'ROA', 'SADC except SA':'SADC',
                                                                'Rest Of Africa':'ROA'})

**Create Learner Citizenship**

In [157]:
cohort_df['Residential Status'] = cohort_df['Residential Status'].fillna('Uknown')
cohort_df['learner_citizenship'] = cohort_df['Residential Status'].map({"Unknown":"U", "South African":"SA",
                                                                        "Permanent Resident":"O", "Dual":"D", "Refugee":"O",
                                                                        "Foreign Nation / Non-Citizen":"O"})


**Create Learner Langauge**

In [158]:
cohort_df['Home Language'] = cohort_df['Home Language'].fillna('Other')
cohort_df['learner_language'] = cohort_df['Home Language'].map({'English':'ENG', 'Afrikaans':'AFR', 'Sepedi [also known as Northern Sotho / Sesotho sa Lebowa]':'SEP',
       'tshiVenda':'TSH', 'isiZulu':'ZUL', 'isiXhosa':'XHO', 'seTswana':'SET', 'xiTsonga':'XIT', 'sePedi':'SEP',
       'isiNdebele':'NDE', 'seSotho':'SES', 'Other':'OTH', 'siSwati':'SWA','seSwati':'SWA','Sesotho':'SES'})


**Create Learner Socio Economic Status**

In [159]:
cohort_df["learner_socioeconomic"] = "01"

**Create Learner Disability**

In [160]:
cohort_df['Disability Status'] = cohort_df['Disability Status'].fillna('Uknown')
cohort_df['learner_disability'] = cohort_df['Disability Status'].map({'No':'N', 'no':'N', 'Yes':'09', 'None':'N', 'yes':'09', 'Yes - specwearer':'01', 'N':'N', 'ADD':'09',
       'Visual impairment - spec wearer':'02', 'Y - specwearer':'02', 'Visual impairment - Spec wearer -':'02', 'Uknown':'U', 'Aspergers / Autistic':'06',
       'Patella Alta':'04', 'yes - specwearer':'02', '\\N':'N', 'Y- specwearer':'02', 'Prefer Not To Say':'U'})


**Create Province Code**

In [161]:
cohort_df['learner_province'] = cohort_df['Home Province'].map({'Gauteng':'8', 'KwaZulu-Natal':'6',
                                                               'Limpopo':'10', 'Western Cape':'2',
                                                               'Eastern Cape':'3', 'Mpumalanga':'9',
                                                               'North West':'7', 'Free State':'5'})

**Create Learner Designation**

In [162]:
cohort_df["learner_designation"] = "Learner"

**Create Learner Company**

In [163]:
cohort_df['learner_company'] = 'WeThinkCode_'

**Create Learner Title**

In [164]:
cohort_df['learner_title'] = ['Mr.' if gender.lower() == 'male' else 'Ms.' for gender in cohort_df['Gender']]

**Create Learner Contact**

In [165]:
cohort_df['learner_contact'] = 'Larissa Brand'

**Create Learner Contact Phone**

In [166]:
cohort_df['learner_contact_phone'] = '082 774 9982'

**Fix Learner Phone**

In [167]:
def convert_phone_to_string(target_df, column):
    """
    Converts phone number column to a string type.
    
    Args:
        target_df: The dataframe containing the column to be converted.
        column: The column to be converted.
        
    Returns:
        N/A
    """
    
    #convert to string
    target_df[column] = target_df[column].astype(str)
    
    #convert to float
    target_df[column] = target_df[column].astype(float)
    
    #convert to int
    target_df[column] = target_df[column].astype(int)
    
    #convert to string
    target_df[column] = target_df[column].astype(str)
    
#convert phone to string
#convert_phone_to_string(cohort_df,"Mobile Number")

**Create Home Address and Physical Address while Studying**

In [168]:
#home address
cohort_df['Home Address'] = cohort_df['Home Street'] + ', ' + cohort_df['Home Suburb'] + ', ' + cohort_df['Home City'] + ', ' + cohort_df['Home Province'] + ', ' + cohort_df['Home Postal Code']
cohort_df['Home Address'] = cohort_df['Home Address'].fillna("Unkown")

#residential address
cohort_df['Residential Address'] = cohort_df['Residential Street'] + ', ' + cohort_df['Residential Suburb'] + ', ' + cohort_df['Residential City'] + ', ' + cohort_df['Residential Province'] + ', ' + cohort_df['Residential Postal Code']
cohort_df['Residential Address'] = cohort_df['Residential Address'].fillna("Unkown")


**Rename Columns**

In [169]:
cohort_df = cohort_df.rename(columns={"ID / Passport Number":"learner_id","Last Name":"learner_surname", "Date Of Birth":"learner_birthdate",
                            "Gender":"learner_gender", "Home Address":"learner_postal_address", "Residential Address":"learner_home_address",
                            "Mobile Phone":"learner_phone", "Email Address":"learner_email"})


**Subset Columns**

In [170]:
#subset columns
column_name = ["learner_name", "learner_surname",'learner_fullname', 'learner_initials',
               'learner_passport','learner_equity', 'learner_nationality', 'learner_citizenship',
               'learner_language', 'learner_socioeconomic', 'learner_disability','learner_province', 'learner_designation',
              "learner_id", "learner_company", "learner_title", "learner_birthdate", "learner_gender", "learner_postal_address",
              "learner_home_address", "learner_phone", "learner_email", "learner_contact", "learner_contact_phone", "Campus"]

cohort_df = cohort_df[column_name]

**Subselect Students**

In [29]:
cohort_df = cohort_df[cohort_df['learner_email'].isin(['llndhlov@student.wethinkcode.co.za'])]

In [44]:
cohort_df.loc[cohort_df['learner_id'] == 'CN806411','learner_citizenship'] = "ZIM"

In [174]:
len(cohort_df)

3

***Send POE Instructions Document***

In [127]:
#create target columns
#poe_instructions_columns = ['learner_fullname', 'learner_id', 'learner_initials', 'learner_company', 'learner_email']

#create template column
#cohort_df['poe_instructions_template_column'] = "feb67241-52f8-456c-917e-bdd0526619ee"

#send poe instructions through signrequests in bulk
#send_bulk_signrequest(cohort_df, poe_instructions_columns, 'poe_instructions_template_column')

Signer: mufaro@thoughtquest.co.za , Status: 201
Signer: mufaro@thoughtquest.co.za , Status: 201
Signer: mufaro@thoughtquest.co.za , Status: 201


**Send Learner Enrolment Contract Document**

In [175]:
#create target columns
#learner_contract_columns = ['learner_title', 'learner_fullname', 'learner_name', 'learner_surname', 'learner_birthdate', 'learner_id', 'learner_passport',
#                           'learner_equity', 'learner_nationality', 'learner_gender', 'learner_citizenship', 'learner_language',
#                           'learner_socioeconomic', 'learner_disability', 'learner_home_address', 'learner_postal_address',
#                           'learner_province', 'learner_phone', 'learner_email', 'learner_company', 'learner_contact', 'learner_contact_phone',
#                           'learner_designation']

#create template column
#cohort_df['learner_contract_template_column'] = '900be1ba-0b96-4386-a904-fb67069f168e'

#send learner contract through signrequests in bulk
#send_bulk_signrequest(cohort_df, learner_contract_columns, 'learner_contract_template_column')


Signer: mufaro@thoughtquest.co.za , Status: 201
Signer: mufaro@thoughtquest.co.za , Status: 201
Signer: mufaro@thoughtquest.co.za , Status: 201


**Send Learner Appeals Procedure**

In [129]:

#create target columns
#learner_appeals_procedure_columns = ['learner_fullname', 'learner_initials']

#create template column
#cohort_df['learner_assessment_contract_template_column'] = '589d226b-1c7e-441f-b99e-1d30a2d37761'

#send learner assessment contract through signrequests in bulk
#send_bulk_signrequest(cohort_df, learner_appeals_procedure_columns, 'learner_assessment_contract_template_column')

Signer: mufaro@thoughtquest.co.za , Status: 201
Signer: mufaro@thoughtquest.co.za , Status: 201
Signer: mufaro@thoughtquest.co.za , Status: 201


**Send Learner Assessment Contract JHB**

In [130]:
#create target columns ****JHB****

#jhb_df = cohort_df[cohort_df['Campus'] == 'Johannesburg']

#learner_assessment_contract_columns = ['learner_fullname', 'learner_id', 'learner_email']

#create template column
#jhb_df['learner_assessment_contract_template_column'] = '215045cd-c510-4d04-8280-ab8dbf0f7d64'

#send learner assessment contract through signrequests in bulk
#send_bulk_signrequest(jhb_df, learner_assessment_contract_columns, 'learner_assessment_contract_template_column')

/Users/assistants/Library/Python/3.8/lib/python/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


Signer: mufaro@thoughtquest.co.za , Status: 201


**Send Learner Assessment COntract CPT**

In [131]:
#create target columns ****CPT****

#cpt_df = cohort_df[cohort_df['Campus'] != 'Johannesburg']

#learner_assessment_contract_columns = ['learner_fullname', 'learner_id', 'learner_email']

#create template column
#cpt_df['learner_assessment_contract_template_column'] = 'aeede3cc-6496-49be-82b0-a959911c522b'

#send learner assessment contract through signrequests in bulk
#send_bulk_signrequest(cpt_df, learner_assessment_contract_columns, 'learner_assessment_contract_template_column')

Signer: mufaro@thoughtquest.co.za , Status: 201
Signer: mufaro@thoughtquest.co.za , Status: 201


**Send Learner PreAssessment Meeting Document JHB**

In [132]:
#create target columns  ****JHB****
#learner_preassessment_document_columns = ['learner_fullname', 'learner_id', 'learner_email']

#create template column
#jhb_df['learner_preassessment_document_template_column'] = '708875f6-b6a8-4876-aa3a-e9104b481ae5'

#send learner preassessment meeting document through signrequests in bulk
#send_bulk_signrequest(jhb_df, learner_preassessment_document_columns, 'learner_preassessment_document_template_column')


Signer: mufaro@thoughtquest.co.za , Status: 201


**Send Learner PreAssessment Meeting Document CPT**

In [133]:
#create target columns  ****JHB****
#learner_preassessment_document_columns = ['learner_fullname', 'learner_id', 'learner_email']

#create template column
#cpt_df['learner_preassessment_document_template_column'] = '08e3cc74-112f-4cd4-ad37-05bc726a5b96'

#send learner preassessment meeting document through signrequests in bulk
#send_bulk_signrequest(cpt_df, learner_preassessment_document_columns, 'learner_preassessment_document_template_column')


Signer: mufaro@thoughtquest.co.za , Status: 201
Signer: mufaro@thoughtquest.co.za , Status: 201


**Send Learner Declaration of Authenticity**

In [134]:
#create target columns
#learner_authenticity_declaration_columns = ['learner_fullname', 'learner_id', 'learner_initials', 'learner_email']

#create template column
#cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'learner_authenticity_declaration_template_column'] = 'd2308911-b247-4957-b865-e76f504ef91f'
#cohort_df.loc[cohort_df['Campus'] != 'Johannesburg', 'learner_authenticity_declaration_template_column'] = '78c858be-3acf-4c37-987d-672536c3196e'

#send learner authenticity document through signrequests in bulk
#send_bulk_signrequest(cohort_df, learner_authenticity_declaration_columns, 'learner_authenticity_declaration_template_column')


**Send Learner Program Evaluation**

In [135]:
#create target columns
#learner_program_evaluation_columns = ['learner_fullname', 'learner_email']

#create template column
#cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'learner_program_evaluation_template_column'] = 'ecae0443-8f3c-44dd-b2ee-7194a4fde67b'
#cohort_df.loc[cohort_df['Campus'] != 'Johannesburg', 'learner_program_evaluation_template_column'] = 'f97eb0a8-97a9-40c7-84df-29aaee7e9893'

#send learner authenticity document through signrequests in bulk
#send_bulk_signrequest(cohort_df, learner_program_evaluation_columns, 'learner_program_evaluation_template_column')


**Send Learner Logbook**

In [136]:
#create target columns
#learner_logbook_columns = ['learner_fullname', 'learner_email', 'learner_initials', 'learner_id']

#create template column
#cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'learner_logbook_template_column'] = 'f8c63d3d-fd0f-413f-a028-35dd9990b30d'
#cohort_df.loc[cohort_df['Campus'] != 'Johannesburg', 'learner_logbook_template_column'] = '911ce292-ec38-4ef9-a8a0-b48b1adbef5f'

#send learner authenticity document through signrequests in bulk
#send_bulk_signrequest(cohort_df, learner_logbook_columns, 'learner_logbook_template_column')


***Create Test Dataframe***

In [173]:
#cohort_df = cohort_df.groupby(['Campus']).head(1).groupby('Campus').head(1).copy()
#cohort_df['learner_email'] = 'mufaro@thoughtquest.co.za'
#send_bulk_signrequest(cohort_df[cohort_df['Campus'] == 'Johannesburg'], learner_preassessment_document_columns, 'learner_preassessment_document_template_column')
